<a href="https://colab.research.google.com/github/JAILBREAK-101/Google_Colab_Project/blob/main/translator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers sentencepiece datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 2

In [2]:
from datasets import load_dataset
from google.colab import drive
from IPython.display import display
from IPython.html import widgets
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import torch
from torch import optim
from torch.nn import functional as F
from transformers import AdamW, AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import get_linear_schedule_with_warmup
from tqdm import tqdm_notebook
import h5py

sns.set()

/usr/local/lib/python3.10/dist-packages/IPython/html.py:12: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  warn("The `IPython.html` package has been deprecated since IPython 4.0. "


In [3]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [39]:
# model_repo = 'google/mt5-base'
# model_path = '/content/gdrive/My Drive/mt5_translation.pt'
# max_seq_len = 20

# model_repo = "google/mt5-small"
# Load model directly
tokenizer = AutoTokenizer.from_pretrained("google/mt5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("google/mt5-small")


In [40]:
# model = model.cuda()
max_sequence_length = model.config.max_length

In [41]:
input_sentence = "Here is our test sentence."
token_ids = tokenizer.encode(input_sentence, return_tensors="pt")

token_ids

model_out = model.generate(token_ids)
print(model_out)

output_text = tokenizer.convert_tokens_to_string(
    tokenizer.convert_ids_to_tokens(model_out[0]))
print(output_text)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


tensor([[     0, 250099,    260,      1]])
<pad> <extra_id_0>.</s>


# Procedures taken
- Load pretrained model and tokenizer
- Load in the dataset
- Transform the dataset into input
- Train/finetune the model on our dataset
- Test the model

In [42]:
example_input_str = "<jp>This is a test sentence."
input_ids = tokenizer.encode(example_input_str, return_tensors="pt")
print("input IDs:", input_ids) # testing for tokenization and vocabulary

tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
print("tokens:", tokens)

input IDs: tensor([[ 1042,  3889,   669, 13673,   339,   259,   262,  2978,   259, 98923,
           260,     1]])
tokens: ['▁<', 'jp', '>', 'This', '▁is', '▁', 'a', '▁test', '▁', 'sentence', '.', '</s>']


In [22]:
sorted(tokenizer.vocab.items(), key=lambda x: x[1])

# Batches of data would be added, thus we are defining the amount of tokens our source and target language would consider


[('<pad>', 0),
 ('</s>', 1),
 ('<unk>', 2),
 ('<0x00>', 3),
 ('<0x01>', 4),
 ('<0x02>', 5),
 ('<0x03>', 6),
 ('<0x04>', 7),
 ('<0x05>', 8),
 ('<0x06>', 9),
 ('<0x07>', 10),
 ('<0x08>', 11),
 ('<0x09>', 12),
 ('<0x0A>', 13),
 ('<0x0B>', 14),
 ('<0x0C>', 15),
 ('<0x0D>', 16),
 ('<0x0E>', 17),
 ('<0x0F>', 18),
 ('<0x10>', 19),
 ('<0x11>', 20),
 ('<0x12>', 21),
 ('<0x13>', 22),
 ('<0x14>', 23),
 ('<0x15>', 24),
 ('<0x16>', 25),
 ('<0x17>', 26),
 ('<0x18>', 27),
 ('<0x19>', 28),
 ('<0x1A>', 29),
 ('<0x1B>', 30),
 ('<0x1C>', 31),
 ('<0x1D>', 32),
 ('<0x1E>', 33),
 ('<0x1F>', 34),
 ('<0x20>', 35),
 ('<0x21>', 36),
 ('<0x22>', 37),
 ('<0x23>', 38),
 ('<0x24>', 39),
 ('<0x25>', 40),
 ('<0x26>', 41),
 ('<0x27>', 42),
 ('<0x28>', 43),
 ('<0x29>', 44),
 ('<0x2A>', 45),
 ('<0x2B>', 46),
 ('<0x2C>', 47),
 ('<0x2D>', 48),
 ('<0x2E>', 49),
 ('<0x2F>', 50),
 ('<0x30>', 51),
 ('<0x31>', 52),
 ('<0x32>', 53),
 ('<0x33>', 54),
 ('<0x34>', 55),
 ('<0x35>', 56),
 ('<0x36>', 57),
 ('<0x37>', 58),
 ('<0x38>',

In [23]:
# Load alt dataset of 13 languages
dataset = load_dataset("alt")

Generating train split:   0%|          | 0/18088 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1019 [00:00<?, ? examples/s]

In [24]:
# Prepare training and test datasets
train_dataset = dataset["train"]
test_dataset = dataset["test"]

In [33]:
# test_dataset
test_dataset[0]

{'SNT.URLID': '78103',
 'SNT.URLID.SNTID': '34',
 'url': 'http://en.wikinews.org/wiki/Horse_flu_infects_Australian_thoroughbreds',
 'translation': {'bg': 'সিডনির র\u200d্যান্ডউইক রেসের মাঠের আটটি থরোব্রেড রেসের ঘোড়া্র এক্যুইন ইনফ্লুয়েঞ্জাতে আক্রান্ত হওয়ার খবরটি নিশ্চিত করা হয়েছে।',
  'en': 'It has been confirmed that eight thoroughbred race horses at Randwick Racecourse in Sydney have been infected with equine influenza.',
  'en_tok': 'It has been confirmed that eight thoroughbred race horses at Randwick Racecourse in Sydney have been infected with equine influenza .',
  'fil': 'Kumpirmado na walong magandang lahing pangkarerang kabayo sa Randwick Racecourse sa Sydney ang nagkaroon ng trangkaso.',
  'hi': 'यह पुष्टि की गई है कि सिडनी में रैंडविक रेसकोर्स के आठ घनी नस्ल के घोड़े इक्वाइन इन्फ्लूएंजा से संक्रमित हैं।',
  'id': 'Telah dikonfirmasi bahwa 8 kuda pacu thoroughbred di pacuan kuda Randwick di Sydney telah terinfeksi oleh flu kuda.',
  'ja': 'シドニーのランドウィック競馬場の8頭のサラブレッド競走馬が馬インフルエン

In [34]:
# train_dataset
train_dataset[0]

{'SNT.URLID': '80188',
 'SNT.URLID.SNTID': '1',
 'url': 'http://en.wikinews.org/wiki/2007_Rugby_World_Cup:_Italy_31_-_5_Portugal',
 'translation': {'bg': 'ফ্রান্সের প্যারিসের পার্ক দি প্রিন্সেস-এ হওয়া ২০০৭-এর রাগবি বিশ্বকাপের পুল সি-তে ইটালি পর্তুগালকে ৩১-৫ গোলে হারিয়েছে।',
  'en': 'Italy have defeated Portugal 31-5 in Pool C of the 2007 Rugby World Cup at Parc des Princes, Paris, France.',
  'en_tok': 'Italy have defeated Portugal 31-5 in Pool C of the 2007 Rugby World Cup at Parc des Princes , Paris , France .',
  'fil': 'Natalo ng Italya ang Portugal sa puntos na 31-5 sa Grupong C noong 2007 sa Pandaigdigang laro ng Ragbi sa Parc des Princes, Paris, France.',
  'hi': '2007 में फ़्रांस, पेरिस के पार्क डेस प्रिंसेस में हुए रग्बी विश्व कप के पूल C में इटली ने पुर्तगाल को 31-5 से हराया।',
  'id': 'Italia berhasil mengalahkan Portugal 31-5 di grup C dalam Piala Dunia Rugby 2007 di Parc des Princes, Paris, Perancis.',
  'ja': 'フランスのパリ、パルク・デ・プランスで行われた2007年ラグビーワールドカップのプールCで、イタリアは31対5でポルトガルを

In [35]:
# PREPARE MY LANGUAGE MAPPINGS FOR DATA PREPROCESSING AND TOKENIZATION
LANG_TOKEN_MAPPING = {
    "bg": "<bg>",
    "en": "<en>",
    "en_tok": "<en_tok>",
    "fil": "<fil>",
    "hi": "<hi>",
    "id": "<id>",
    "ja": "<ja>",
    "khm": "<khm>",
    "lo": "<lo>",
    "ms": "<ms>",
    "my": "<my>",
    "th": "<th>",
    "vi": "<vi>",
    "zh": "<zh>"
}


In [36]:
# Add special tokens to the dictionary
special_tokens_dictionary = {
    "additional_special_tokens": list(LANG_TOKEN_MAPPING.values())
}

tokenizer.add_special_tokens(special_tokens_dictionary)

model.resize_token_embeddings(len(tokenizer))

Embedding(250113, 512)

In [45]:
# Create the encoder using the token ids of the sequence of texts
token_ids = tokenizer.encode(
    example_input_str,
    return_tensors="pt",
    padding="max_length",
    truncation=True,
    max_length=max_sequence_length
)
print(token_ids)

tensor([[ 1042,  3889,   669, 13673,   339,   259,   262,  2978,   259, 98923,
           260,     1,     0,     0,     0,     0,     0,     0,     0,     0]])


In [48]:
# The Encoder Function for the translator

def encode_input_str(text, target_lang, tokenizer, seq_len,
                     lang_token_map=LANG_TOKEN_MAPPING):
  target_lang_token = lang_token_map[target_lang]

  # Tokenize and add special tokens
  input_ids = tokenizer.encode(
      text = target_lang_token + text,
      return_tensors = 'pt',
      padding = 'max_length',
      truncation = True,
      max_length = seq_len)

  return input_ids[0]

In [49]:
def encode_target_str(text, tokenizer, seq_len,
                      lang_token_map=LANG_TOKEN_MAPPING):
  token_ids = tokenizer.encode(
      text = text,
      return_tensors = 'pt',
      padding = 'max_length',
      truncation = True,
      max_length = seq_len)

  return token_ids[0]

In [50]:
def format_translation_data(translations, lang_token_map,
                            tokenizer, seq_len=128):
  # Choose a random 2 languages for in i/o
  langs = list(lang_token_map.keys())
  input_lang, target_lang = np.random.choice(langs, size=2, replace=False)

  # Get the translations for the batch
  input_text = translations[input_lang]
  target_text = translations[target_lang]

  if input_text is None or target_text is None:
    return None

  input_token_ids = encode_input_str(
      input_text, target_lang, tokenizer, seq_len, lang_token_map)

  target_token_ids = encode_target_str(
      target_text, tokenizer, seq_len, lang_token_map)

  return input_token_ids, target_token_ids


In [81]:
def transform_batch(batch, lang_token_map, tokenizer):
  inputs = []
  targets = []
  for translation_set in batch['translation']:
    formatted_data = format_translation_data(
        translation_set, lang_token_map, tokenizer, max_sequence_length)

    if formatted_data is None:
      continue

    input_ids, target_ids = formatted_data
    inputs.append(input_ids.unsqueeze(0))
    targets.append(target_ids.unsqueeze(0))

  # batch_input_ids = torch.cat(inputs).cuda()
  # batch_target_ids = torch.cat(targets).cuda()

  batch_input_ids = torch.cat(inputs)
  batch_target_ids = torch.cat(targets)

  return batch_input_ids, batch_target_ids

In [82]:
# Format the data for batches

def transform_batch(batch,  lang_token_map, tokenizer):
  inputs = []
  targets = []
  for translation_set in batch['translation']:
    formatted_data = format_translation_data(
        translation_set, lang_token_map, tokenizer, max_sequence_length)

    if formatted_data is None:
      continue

    input_ids, target_ids = formatted_data
    inputs.append(input_ids.unsqueeze(0))
    targets.append(target_ids.unsqueeze(0))

  batch_input_ids = torch.cat(inputs).cuda()
  batch_target_ids = torch.cat(targets).cuda()

  return batch_input_ids, batch_target_ids

In [83]:
def get_data_generator(dataset, lang_token_map, tokenizer, batch_size=32):
  dataset = dataset.shuffle()
  for i in range(0, len(dataset), batch_size):
    raw_batch = dataset[i:i+batch_size]
    yield transform_batch(raw_batch, lang_token_map, tokenizer)

In [84]:
input_ids, output_ids = format_translation_data(
    train_dataset[1]['translation'], LANG_TOKEN_MAPPING, tokenizer)

print(' '.join(tokenizer.convert_ids_to_tokens(input_ids)))
print(' '.join(tokenizer.convert_ids_to_tokens(output_ids)))

data_generator = get_data_generator(train_dataset, LANG_TOKEN_MAPPING, tokenizer, 8)
# data_batch = next(data_generator)
# print("Input shape: ", data_batch[0].shape)
# print("Output shape: ", data_batch[1].shape)
# Because of the limitation of NVIDIA Graphics Card on this system,No cuda

▁< ja > အ န် ဒ ရီ ယာ ▁ မာ စီ ▁ သည် ▁အီ တ လီ ▁ အတွက် ▁စ မ်း သ ပ် မှု ▁ တစ်ခု ▁ အဖြစ် ▁စ တ ုတ္ ထ ▁ မိ နစ် ▁ တွင် ▁ အမှတ် ပေး ခြင်း ▁ ကို ▁ဖ ွင် ့ လှ စ် ပေး ခဲ့ သည် ▁ ။ </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
▁ アンド レア ・ マー ジ が 開始 4 分 後の トライ で イタリア にとって 最初の 得点 を入れ た 。 </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pa

# MODEL TRAINING AND FINE-TUNING

In [85]:
# Making use of epochs

# defining the model training parameters
n_epochs = 5
batch_size = 16
print_freq = 50
lr = 5e-4
n_batches = int(np.ceil(len(train_dataset) / batch_size))
total_steps = n_epochs * n_batches
n_warmup_steps = int(0.01 * total_steps)

In [86]:
# Optimizer mt5
optimizer = AdamW(model.parameters(), lr=lr)

scheduler = get_linear_schedule_with_warmup(optimizer,n_warmup_steps,total_steps)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [88]:
losses = []

In [89]:
for epoch_idx in range(n_epochs):
  # randomize data order
  data_generator = get_data_generator(train_dataset, LANG_TOKEN_MAPPING, tokenizer, batch_size)

  for batch_idx, (input_batch, label_batch) \
      in enumerate(tqdm_notebook(data_generator, total=n_batches)):

    optimizer.zero_grad()

    # forward pass
    model_out = model.forward(
        input_ids = input_batch,
        labels = label_batch)

    loss = model_out.loss
    losses.append(loss.item())

    # backward pass
    loss.backward()
    optimizer.step()
    scheduler.step()

    if (batch_idx + 1) % print_freq == 0:
      avg_loss = np.mean(losses[-print_freq:])
      print('Epoch: {} | Step: {} | Avg .loss: {:.3f} | lr: {}'.format(
          epoch_idx + 1, batch_idx + 1, avg_loss, scheduler.get_last_lr()[0]))

<ipython-input-89-eb8e28c182e4>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  in enumerate(tqdm_notebook(data_generator, total=n_batches)):


  0%|          | 0/1131 [00:00<?, ?it/s]

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx